### Carregando bibliotecas

In [1]:
import numpy as np
import pandas as pd
import sklearn
import json
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics

from sklearn.model_selection import KFold 
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion
from scipy.sparse import coo_matrix, hstack, vstack
from sklearn_pandas import DataFrameMapper

### Importanto e tratando os datasets de cada pais

In [2]:
CAvideos = pd.read_csv('CAvideos.csv')
DEvideos = pd.read_csv('DEvideos.csv')
FRvideos = pd.read_csv('FRvideos.csv')
GBvideos = pd.read_csv('GBvideos.csv')
USvideos = pd.read_csv('USvideos.csv')

In [3]:
print("CA DataSet shape: "+str(CAvideos.shape))
print("DE DataSet shape: "+str(DEvideos.shape))
print("FR DataSet shape: "+str(FRvideos.shape))
print("GB DataSet shape: "+str(GBvideos.shape))
print("US DataSet shape: "+str(USvideos.shape))

CA DataSet shape: (40881, 16)
DE DataSet shape: (40840, 16)
FR DataSet shape: (40724, 16)
GB DataSet shape: (38916, 16)
US DataSet shape: (40949, 16)


In [4]:
#print("CA DataSet null inputs: \n"+str(CAvideos.isnull().sum())+"\n")
#print("DE DataSet null inputs: \n"+str(DEvideos.isnull().sum())+"\n")
#print("FR DataSet null inputs: \n"+str(FRvideos.isnull().sum())+"\n")
#print("GB DataSet null inputs: \n"+str(GBvideos.isnull().sum())+"\n")
print("US DataSet null inputs: \n"+str(USvideos.isnull().sum())+"\n")

US DataSet null inputs: 
video_id                    0
trending_date               0
title                       0
channel_title               0
category_id                 0
publish_time                0
tags                        0
views                       0
likes                       0
dislikes                    0
comment_count               0
thumbnail_link              0
comments_disabled           0
ratings_disabled            0
video_error_or_removed      0
description               570
dtype: int64



In [5]:
CAvideos = CAvideos.dropna(axis=0)
DEvideos = DEvideos.dropna(axis=0)
FRvideos = FRvideos.dropna(axis=0)
GBvideos = GBvideos.dropna(axis=0)
USvideos = USvideos.dropna(axis=0)

In [6]:
#print("CA DataSet null inputs: \n"+str(CAvideos.isnull().sum())+"\n")
#print("DE DataSet null inputs: \n"+str(DEvideos.isnull().sum())+"\n")
#print("FR DataSet null inputs: \n"+str(FRvideos.isnull().sum())+"\n")
#print("GB DataSet null inputs: \n"+str(GBvideos.isnull().sum())+"\n")
print("US DataSet null inputs: \n"+str(USvideos.isnull().sum())+"\n")

US DataSet null inputs: 
video_id                  0
trending_date             0
title                     0
channel_title             0
category_id               0
publish_time              0
tags                      0
views                     0
likes                     0
dislikes                  0
comment_count             0
thumbnail_link            0
comments_disabled         0
ratings_disabled          0
video_error_or_removed    0
description               0
dtype: int64



In [7]:
#print("CA DataSet shape: "+str(CAvideos.shape))
#print("DE DataSet shape: "+str(DEvideos.shape))
#print("FR DataSet shape: "+str(FRvideos.shape))
#print("GB DataSet shape: "+str(GBvideos.shape))
print("US DataSet shape: "+str(USvideos.shape))

US DataSet shape: (40379, 16)


## Correção de formato de data-hora para padrão Unix

In [8]:
USvideos['trending_date'] = pd.to_datetime(USvideos['trending_date'], format='%y.%d.%m')
USvideos['publish_time'] = pd.to_datetime(USvideos['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

# separates date and time into two columns from 'publish_time' column

USvideos.insert(4, 'publish_date', USvideos['publish_time'].dt.date)
USvideos['publish_time'] = USvideos['publish_time'].dt.time
USvideos['publish_date']=pd.to_datetime(USvideos['publish_date'])

In [9]:
USvideos.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,2017-11-13,22,17:13:01,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,2017-11-13,24,07:30:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,2017-11-12,23,19:05:24,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,2017-11-13,24,11:00:04,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,2017-11-12,24,18:01:41,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


## Tratando para pegar somente a ultima entrada para cada video (a entrada mais atualizada)

In [10]:
USvideos_lastentry = USvideos.sort_values('trending_date', ascending=False).drop_duplicates(['video_id'])
USvideos_lastentry.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
40948,ooyjaVdt-jA,2018-06-14,Official Call of Duty®: Black Ops 4 — Multipla...,Call of Duty,2018-05-17,20,17:09:38,"call of duty|""cod""|""activision""|""Black Ops 4""",10306119,357079,212976,144795,https://i.ytimg.com/vi/ooyjaVdt-jA/default.jpg,False,False,False,Call of Duty: Black Ops 4 Multiplayer raises t...
40811,Nd3zqXro_P0,2018-06-14,Why 350°F is the magic number for baking,Vox,2018-06-07,25,12:00:02,"baking|""maillard reaction""|""s pen""|""Vox.com""|""...",799411,16092,652,1551,https://i.ytimg.com/vi/Nd3zqXro_P0/default.jpg,False,False,False,Turns out there’s a lot of chemistry in cookin...
40821,jWkOwC7vMb4,2018-06-14,“Bonding” with Grandma,itsAlexClark,2018-06-06,1,21:40:34,"its alex clark|""itsalexclark""|""alex clark""|""it...",1004181,48129,889,5483,https://i.ytimg.com/vi/jWkOwC7vMb4/default.jpg,False,False,False,"You know my family, now you need to meet my gr..."
40820,T9UHD30MSGk,2018-06-14,Knife Expert Guesses Cheap vs. Expensive Knive...,Epicurious,2018-06-06,26,18:54:38,"best knife|""boning knife""|""cheap""|""cheap vs ex...",903651,11427,2305,2504,https://i.ytimg.com/vi/T9UHD30MSGk/default.jpg,False,False,False,Epicurious challenges knife expert Geoff Feder...
40819,NB3gWkhLkxM,2018-06-14,The Internet - Come Over (Official Video),TheInternetVEVO,2018-06-06,10,14:00:02,"Columbia|""Come Over""|""R&B/Soul""|""The Internet""",1615678,92403,1856,4572,https://i.ytimg.com/vi/NB3gWkhLkxM/default.jpg,False,False,False,The Internet's Come Over is taken from the new...


In [11]:
print("US DataSet shape: "+str(USvideos_lastentry.shape))

US DataSet shape: (6254, 17)


## Buscando o nome da categoria pelo identificador e armazenando em nova coluna

In [12]:
with open('US_category_id.json') as f:
    UScategory = json.load(f)

In [13]:
def category_replace(c_id):
    for i in UScategory["items"]:
        if int(c_id) == int(i["id"]):
             return i["snippet"]["title"]
    return "None"

In [14]:
USvideos_lastentry['category_name'] = USvideos_lastentry.apply(lambda row: category_replace(row['category_id']), axis=1)

In [15]:
USvideos_lastentry.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name
40948,ooyjaVdt-jA,2018-06-14,Official Call of Duty®: Black Ops 4 — Multipla...,Call of Duty,2018-05-17,20,17:09:38,"call of duty|""cod""|""activision""|""Black Ops 4""",10306119,357079,212976,144795,https://i.ytimg.com/vi/ooyjaVdt-jA/default.jpg,False,False,False,Call of Duty: Black Ops 4 Multiplayer raises t...,Gaming
40811,Nd3zqXro_P0,2018-06-14,Why 350°F is the magic number for baking,Vox,2018-06-07,25,12:00:02,"baking|""maillard reaction""|""s pen""|""Vox.com""|""...",799411,16092,652,1551,https://i.ytimg.com/vi/Nd3zqXro_P0/default.jpg,False,False,False,Turns out there’s a lot of chemistry in cookin...,News & Politics
40821,jWkOwC7vMb4,2018-06-14,“Bonding” with Grandma,itsAlexClark,2018-06-06,1,21:40:34,"its alex clark|""itsalexclark""|""alex clark""|""it...",1004181,48129,889,5483,https://i.ytimg.com/vi/jWkOwC7vMb4/default.jpg,False,False,False,"You know my family, now you need to meet my gr...",Film & Animation
40820,T9UHD30MSGk,2018-06-14,Knife Expert Guesses Cheap vs. Expensive Knive...,Epicurious,2018-06-06,26,18:54:38,"best knife|""boning knife""|""cheap""|""cheap vs ex...",903651,11427,2305,2504,https://i.ytimg.com/vi/T9UHD30MSGk/default.jpg,False,False,False,Epicurious challenges knife expert Geoff Feder...,Howto & Style
40819,NB3gWkhLkxM,2018-06-14,The Internet - Come Over (Official Video),TheInternetVEVO,2018-06-06,10,14:00:02,"Columbia|""Come Over""|""R&B/Soul""|""The Internet""",1615678,92403,1856,4572,https://i.ytimg.com/vi/NB3gWkhLkxM/default.jpg,False,False,False,The Internet's Come Over is taken from the new...,Music


## Definindo sentimento por video

In [16]:
#positive sentiment = 60% da quantidade de likes+dislikes
USvideos_lastentry['sentiment'] = USvideos_lastentry['likes'] >= 0.6*(USvideos_lastentry['likes']+USvideos_lastentry['dislikes']) 
USvideos_lastentry.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment
40948,ooyjaVdt-jA,2018-06-14,Official Call of Duty®: Black Ops 4 — Multipla...,Call of Duty,2018-05-17,20,17:09:38,"call of duty|""cod""|""activision""|""Black Ops 4""",10306119,357079,212976,144795,https://i.ytimg.com/vi/ooyjaVdt-jA/default.jpg,False,False,False,Call of Duty: Black Ops 4 Multiplayer raises t...,Gaming,True
40811,Nd3zqXro_P0,2018-06-14,Why 350°F is the magic number for baking,Vox,2018-06-07,25,12:00:02,"baking|""maillard reaction""|""s pen""|""Vox.com""|""...",799411,16092,652,1551,https://i.ytimg.com/vi/Nd3zqXro_P0/default.jpg,False,False,False,Turns out there’s a lot of chemistry in cookin...,News & Politics,True
40821,jWkOwC7vMb4,2018-06-14,“Bonding” with Grandma,itsAlexClark,2018-06-06,1,21:40:34,"its alex clark|""itsalexclark""|""alex clark""|""it...",1004181,48129,889,5483,https://i.ytimg.com/vi/jWkOwC7vMb4/default.jpg,False,False,False,"You know my family, now you need to meet my gr...",Film & Animation,True
40820,T9UHD30MSGk,2018-06-14,Knife Expert Guesses Cheap vs. Expensive Knive...,Epicurious,2018-06-06,26,18:54:38,"best knife|""boning knife""|""cheap""|""cheap vs ex...",903651,11427,2305,2504,https://i.ytimg.com/vi/T9UHD30MSGk/default.jpg,False,False,False,Epicurious challenges knife expert Geoff Feder...,Howto & Style,True
40819,NB3gWkhLkxM,2018-06-14,The Internet - Come Over (Official Video),TheInternetVEVO,2018-06-06,10,14:00:02,"Columbia|""Come Over""|""R&B/Soul""|""The Internet""",1615678,92403,1856,4572,https://i.ytimg.com/vi/NB3gWkhLkxM/default.jpg,False,False,False,The Internet's Come Over is taken from the new...,Music,True


## Normalização das features numéricas

In [17]:
USvideos_lastentry.describe()

,category_id,views,likes,dislikes,comment_count
count,6254.000000,6.254000e+03,6.254000e+03,6.254000e+03,6.254000e+03
mean,20.408539,1.975222e+06,5.592469e+04,2.828081e+03,6.289861e+03
std,7.256811,7.092548e+06,1.942315e+05,2.433277e+04,2.943713e+04
min,1.000000,5.590000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,1.648875e+05,2.931000e+03,1.300000e+02,3.870000e+02
50%,24.000000,5.270375e+05,1.222400e+04,4.485000e+02,1.287500e+03
75%,25.000000,1.490734e+06,3.898500e+04,1.516000e+03,4.110750e+03
max,43.000000,2.252119e+08,5.613827e+06,1.643059e+06,1.228655e+06


In [18]:
# Normalização
USvideos_lastentry['views'] = (USvideos_lastentry['views']-USvideos_lastentry['views'].min())/(USvideos_lastentry['views'].max()-USvideos_lastentry['views'].min())
USvideos_lastentry['likes'] = (USvideos_lastentry['likes']-USvideos_lastentry['likes'].min())/(USvideos_lastentry['likes'].max()-USvideos_lastentry['likes'].min())
USvideos_lastentry['dislikes'] = (USvideos_lastentry['dislikes']-USvideos_lastentry['dislikes'].min())/(USvideos_lastentry['dislikes'].max()-USvideos_lastentry['dislikes'].min())
USvideos_lastentry['comment_count'] = (USvideos_lastentry['comment_count']-USvideos_lastentry['comment_count'].min())/(USvideos_lastentry['comment_count'].max()-USvideos_lastentry['comment_count'].min())


In [19]:
USvideos_lastentry.describe()

,category_id,views,likes,dislikes,comment_count
count,6254.000000,6254.000000,6254.000000,6254.000000,6254.000000
mean,20.408539,0.008768,0.009962,0.001721,0.005119
std,7.256811,0.031493,0.034599,0.014809,0.023959
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,0.000730,0.000522,0.000079,0.000315
50%,24.000000,0.002338,0.002177,0.000273,0.001048
75%,25.000000,0.006617,0.006944,0.000923,0.003346
max,43.000000,1.000000,1.000000,1.000000,1.000000


## Separando conjuntos de treino e teste

In [20]:
train_data = USvideos_lastentry.sample(frac=0.8, random_state=200)
test_data  = USvideos_lastentry.drop(train_data.index)
print(train_data.shape, test_data.shape)

(5003, 19) (1251, 19)


## Bag of words Title

In [21]:
count_vectorizer = CountVectorizer()
x_train_title = count_vectorizer.fit_transform(train_data['title'].values)
y_train_title = train_data['sentiment'].values

In [22]:
classifier_title = LogisticRegression()
classifier_title.fit(x_train_title, y_train_title)

c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [23]:
x_test_title = count_vectorizer.transform(test_data['title'].values)
y_test_title = test_data['sentiment'].values
predicted = classifier_title.predict(x_test_title)

In [24]:
get_n_features = 5
print("The "+str(get_n_features)+" most positive-weighted words are: ")
for feature_id in sorted(range(len(classifier_title.coef_[0])), key=lambda i: classifier_title.coef_[0][i], reverse=True)[:get_n_features]:
    print("%s (%f)" % (count_vectorizer.get_feature_names()[feature_id], classifier_title.coef_[0][feature_id]))
print()
print("The "+str(get_n_features)+" most negative-weighted words are: ")
for feature_id in sorted(range(len(classifier_title.coef_[0])), key=lambda i: classifier_title.coef_[0][i], reverse=False)[:get_n_features]:
    print("%s (%f)" % (count_vectorizer.get_feature_names()[feature_id], classifier_title.coef_[0][feature_id]))


The 5 most positive-weighted words are: 
vs (1.009870)
how (0.966596)
it (0.952246)
best (0.777220)
look (0.750661)

The 5 most negative-weighted words are: 
trump (-1.931192)
metoo (-1.735375)
january (-1.504494)
clinton (-1.480359)
fbi (-1.385043)


In [25]:
probs = classifier_title.predict_proba(x_test_title)
print(probs)

[[0.01415181 0.98584819]
 [0.01564468 0.98435532]
 [0.00688497 0.99311503]
 ...
 [0.00268434 0.99731566]
 [0.01390236 0.98609764]
 [0.01025454 0.98974546]]


In [26]:
print(metrics.accuracy_score(y_test_title, predicted))
print(metrics.roc_auc_score(y_test_title, probs[:, 1]))

0.9688249400479616
0.7370373584414458


In [27]:
probs = classifier_title.predict_proba(count_vectorizer.transform(USvideos_lastentry['title']))

In [28]:
USvideos_title = USvideos_lastentry

In [29]:
USvideos_title['predicted_sentiment'] = probs.max(axis=1)

In [30]:
USvideos_title = USvideos_title.sort_values('predicted_sentiment', ascending=False)
USvideos_title.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment,predicted_sentiment
27744,B2W7c3XRBwQ,2018-04-03,How To Make a Easter GIANT PEEP CAKE | With Ye...,How To Cake It,2018-03-20,26,15:00:19,"Google|""YouTube""|""Yolanda Gampp""|""Yolanda Gamp...",0.002954,0.003685,0.000215,0.002357,https://i.ytimg.com/vi/B2W7c3XRBwQ/default.jpg,False,False,False,My Cakebook Is Available WORLDWIDE! http://bit...,Howto & Style,True,0.999791
4386,QgaYsdnOMiM,2017-12-05,How To Make A GIANT CANDY CANE Cake w. SURPRIS...,How To Cake It,2017-11-28,26,16:00:38,"dunkin donuts|""Yolanda Gampp""|""giant cakes""|""p...",0.002843,0.004562,0.000189,0.002123,https://i.ytimg.com/vi/QgaYsdnOMiM/default.jpg,False,False,False,Register for Camp Cake & WIN a Bundle! - http:...,Howto & Style,True,0.999774
30435,DTg2FihPSso,2018-04-23,Stephen A.: Kevin Hart 'got his feelings hurt'...,ESPN,2018-04-17,17,14:55:31,"espn|""dwyane wade""|""dwayne wade""|""d wade""|""76e...",0.004487,0.001238,0.000350,0.001240,https://i.ytimg.com/vi/DTg2FihPSso/default.jpg,False,False,False,First Take's Stephen A. Smith says Kevin Hart ...,Sports,True,0.999750
15514,iNT-tPrz4OM,2018-02-01,Footage of how Tom Cruise broke his ankle whil...,BBC,2018-01-26,24,19:03:33,"BBC|""iPlayer""|""Graham Norton""|""Tom Cruise""|""Si...",0.002002,0.000642,0.000091,0.000448,https://i.ytimg.com/vi/iNT-tPrz4OM/default.jpg,False,False,False,Programme website: http://bbc.in/2FkwhuK Tom C...,Entertainment,True,0.999744
31349,-EL8TuMsb-k,2018-04-27,Full Face TESTING BEAUTY GURUS Makeup FAVORITE...,Sylvia Gani,2018-04-09,26,00:07:02,"FULL FACE|""full face using""|""full face testing...",0.002508,0.003724,0.000257,0.001836,https://i.ytimg.com/vi/-EL8TuMsb-k/default.jpg,False,False,False,SUBSCRIBE HERE: http://bit.ly/2g1cqFW\nHey S-C...,Howto & Style,True,0.999689


## Bag of words Descrição

In [31]:
count_vectorizer = CountVectorizer(stop_words='english')
x_train_desc = count_vectorizer.fit_transform(train_data['description'].values)
y_train_desc = train_data['sentiment'].values

In [32]:
classifier_desc = LogisticRegression()
classifier_desc.fit(x_train_desc, y_train_desc)

c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [33]:
x_test_desc = count_vectorizer.transform(test_data['description'].values)
y_test_desc = test_data['sentiment'].values
predicted = classifier_desc.predict(x_test_desc)

In [34]:
get_n_features = 5
print("The "+str(get_n_features)+" most positive-weighted words are: ")
for feature_id in sorted(range(len(classifier_desc.coef_[0])), key=lambda i: classifier_desc.coef_[0][i], reverse=True)[:get_n_features]:
    print("%s (%f)" % (count_vectorizer.get_feature_names()[feature_id], classifier_desc.coef_[0][feature_id]))
print()
print("The "+str(get_n_features)+" most negative-weighted words are: ")
for feature_id in sorted(range(len(classifier_desc.coef_[0])), key=lambda i: classifier_desc.coef_[0][i], reverse=False)[:get_n_features]:
    print("%s (%f)" % (count_vectorizer.get_feature_names()[feature_id], classifier_desc.coef_[0][feature_id]))


The 5 most positive-weighted words are: 
ellen (0.781361)
tuesday (0.657431)
thanksgiving (0.511212)
larry (0.503538)
performance (0.502253)

The 5 most negative-weighted words are: 
trump (-1.485354)
speaking (-1.059645)
variety (-0.871246)
women (-0.762987)
crossfit (-0.731848)


In [35]:
probs = classifier_desc.predict_proba(x_test_desc)
print(probs)

[[2.01555257e-05 9.99979844e-01]
 [8.11684769e-02 9.18831523e-01]
 [1.21249411e-09 9.99999999e-01]
 ...
 [1.24382423e-03 9.98756176e-01]
 [2.18114898e-03 9.97818851e-01]
 [7.79182598e-04 9.99220817e-01]]


In [36]:
print(metrics.accuracy_score(y_test_desc, predicted))
print(metrics.roc_auc_score(y_test_desc, probs[:, 1]))

0.9680255795363709
0.7995400702911443


In [37]:
probs = classifier_desc.predict_proba(count_vectorizer.transform(USvideos_lastentry['description']))

In [38]:
USvideos_description = USvideos_lastentry

In [39]:
USvideos_description['predicted_sentiment'] = probs.max(axis=1)

In [40]:
USvideos_description = USvideos_description.sort_values('predicted_sentiment', ascending=False)
USvideos_description.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment,predicted_sentiment
2591,PliRMGvytnA,2017-11-26,Drug Store Nail Powders FAIL (what the Sally H...,Simply Nailogical,2017-11-18,24,16:43:23,"nails|""nail art""|""nail tutorial""|""beauty tutor...",0.010282,0.017198,0.000662,0.008225,https://i.ytimg.com/vi/PliRMGvytnA/default.jpg,False,False,False,When the thumbnail isn't clickbait ¯\_(ツ)_/¯ #...,Entertainment,True,1.0
20154,b03U6BYF9L0,2018-02-24,Debunking Anti-Vaxxers,AsapSCIENCE,2018-02-15,28,17:09:04,"anti-vaxxer|""why you should get vaccinated""|""w...",0.003755,0.009106,0.001608,0.011547,https://i.ytimg.com/vi/b03U6BYF9L0/default.jpg,False,False,False,Here's what to say to anti-vaxxers!\nCheck out...,Science & Technology,True,1.0
6391,23QOy9Q2qNI,2017-12-15,Best Videos of the Year 2017 | People Are Awesome,People are Awesome,2017-12-08,24,13:01:46,"people are awesome|""people are awesome 2017""|""...",0.004378,0.002705,0.000121,0.000410,https://i.ytimg.com/vi/23QOy9Q2qNI/default.jpg,False,False,False,Here are the very best videos we featured in 2...,Entertainment,True,1.0
14376,lUZPV3gGXgY,2018-01-26,Magic Transforming Top Coat?! (does this thing...,Simply Nailogical,2018-01-20,24,15:57:00,"nails|""nail art""|""nail tutorial""|""beauty tutor...",0.009782,0.021877,0.000584,0.011045,https://i.ytimg.com/vi/lUZPV3gGXgY/default.jpg,False,False,False,Prepare yourself for this is an actual nail vi...,Entertainment,True,1.0
35879,iGD4bBgWPcY,2018-05-20,[BEHIND THE SCENES] Havana – Pentatonix,PTXofficial,2018-05-08,10,14:02:11,"Pentatonix|""Penatonix""|""PTX""|""PTXofficial""|""Mi...",0.002371,0.003275,0.000287,0.000715,https://i.ytimg.com/vi/iGD4bBgWPcY/default.jpg,False,False,False,"PTX PRESENTS: TOP POP, VOL. I OUT NOW FEATURIN...",Music,True,1.0


## Bag of words Tag

In [41]:
count_vectorizer = CountVectorizer()
x_train_tag = count_vectorizer.fit_transform(train_data['tags'].values)
y_train_tag = train_data['sentiment'].values

In [42]:
classifier_tag = LogisticRegression()
classifier_tag.fit(x_train_tag, y_train_tag)

c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [43]:
x_test_tag = count_vectorizer.transform(test_data['tags'].values)
y_test_tag = test_data['sentiment'].values
predicted = classifier_tag.predict(x_test_tag)

In [44]:
get_n_features = 5
print("The "+str(get_n_features)+" most positive-weighted words are: ")
for feature_id in sorted(range(len(classifier_tag.coef_[0])), key=lambda i: classifier_tag.coef_[0][i], reverse=True)[:get_n_features]:
    print("%s (%f)" % (count_vectorizer.get_feature_names()[feature_id], classifier_tag.coef_[0][feature_id]))
print()
print("The "+str(get_n_features)+" most negative-weighted words are: ")
for feature_id in sorted(range(len(classifier_tag.coef_[0])), key=lambda i: classifier_tag.coef_[0][i], reverse=False)[:get_n_features]:
    print("%s (%f)" % (count_vectorizer.get_feature_names()[feature_id], classifier_tag.coef_[0][feature_id]))


The 5 most positive-weighted words are: 
2017 (0.846641)
records (0.804237)
world (0.785464)
pop (0.720362)
thanksgiving (0.631710)

The 5 most negative-weighted words are: 
politics (-1.082239)
heist (-1.032381)
animatronic (-0.997861)
variety (-0.956415)
none (-0.947209)


In [45]:
probs = classifier_tag.predict_proba(x_test_tag)
print(probs)

[[8.44070815e-04 9.99155929e-01]
 [2.80081001e-03 9.97199190e-01]
 [6.68810880e-05 9.99933119e-01]
 ...
 [1.04410653e-02 9.89558935e-01]
 [4.53253185e-04 9.99546747e-01]
 [6.37753368e-04 9.99362247e-01]]


In [46]:
print(metrics.accuracy_score(y_test_tag, predicted))
print(metrics.roc_auc_score(y_test_tag, probs[:, 1]))

0.9688249400479616
0.8171237037358441


In [47]:
probs = classifier_tag.predict_proba(count_vectorizer.transform(USvideos_lastentry['tags']))

In [48]:
USvideos_tags = USvideos_lastentry

In [49]:
USvideos_tags['predicted_sentiment'] = probs.max(axis=1)

In [50]:
USvideos_tags = USvideos_tags.sort_values('predicted_sentiment', ascending=False)
USvideos_tags.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment,predicted_sentiment
10797,pKFd5rAUM2s,2018-01-06,BLANK CANVAS... 2017 YEAR IN REVIEW,James Charles,2017-12-28,24,19:00:22,"james|""james charles""|""charles""|""makeup""|""mua""...",0.011210,0.040454,0.002962,0.013636,https://i.ytimg.com/vi/pKFd5rAUM2s/default.jpg,False,False,False,HI SISTERS! 2017 is coming to a close and it's...,Entertainment,True,1.0
7199,wwX-RSqxEXc,2017-12-19,17 of 2017: The Year in Pop Culture (ft. Damon...,Kingsley,2017-12-13,24,20:30:03,"kingsley|""really b really""|""kingsley really b ...",0.000125,0.000452,0.000004,0.000156,https://i.ytimg.com/vi/wwX-RSqxEXc/default.jpg,False,False,False,Give this vid a thumbs up and SUBSCRIBE : http...,Entertainment,True,1.0
11194,mDRifR9HdK8,2018-01-08,Best Pet Videos of the Year 2017!,The Pet Collective,2017-12-30,15,14:00:04,"the pet collective|""pet collective""|""the pet c...",0.003703,0.002674,0.000369,0.000647,https://i.ytimg.com/vi/mDRifR9HdK8/default.jpg,False,False,False,2017 has been quite the year and I don't know ...,Pets & Animals,True,1.0
7777,ukqJTaHm-xc,2017-12-22,Worst & Weirdest Beauty Trends of 2017!! (Beau...,Clevver Style,2017-12-16,26,16:04:11,"2017|""beauty break""|""beauty""|""beauty trends 20...",0.001912,0.002034,0.000194,0.000389,https://i.ytimg.com/vi/ukqJTaHm-xc/default.jpg,False,False,False,VOTE FOR OUR PROJECT FOR AWESOME VIDEO! ►► htt...,Howto & Style,True,1.0
8796,4Cc-U8uhXYk,2017-12-27,THE HAIR COLOR THAT WILL BEST SUIT YOU AND YOU...,Brad Mondo,2017-12-18,27,22:56:00,HAIR COLOR THAT WILL BEST SUIT YOU AND YOUR SK...,0.001860,0.002805,0.000265,0.002014,https://i.ytimg.com/vi/4Cc-U8uhXYk/default.jpg,False,False,False,Here is a guide to picking the perfect hair co...,Education,True,1.0


In [51]:
def run_kfold(clf,X,Y):
    kf = KFold(n_splits=10)
    outcomes = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        outcomes.append(accuracy)
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

In [52]:
#Media dos Score para Title 
run_kfold(classifier_title,x_train_title,y_train_title)

Mean Accuracy: 0.9722191616766468


c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [53]:
#Media dos Score para Descrição
run_kfold(classifier_desc,x_train_desc,y_train_desc)

Mean Accuracy: 0.9692215568862277


In [54]:
#Media dos Score para Tags
run_kfold(classifier_tag,x_train_tag,y_train_tag)

Mean Accuracy: 0.9696203592814371


In [55]:
#combined_features = FeatureUnion([('CountTitle', x_train_title),('CountDesc', x_train_desc), ('CountTag', x_train_tag)])

In [56]:
def using_nonzero(x):
    rows,cols = x.nonzero()
    for row,col in zip(rows,cols):
        print(((row,col), x[row,col]))

In [57]:
using_nonzero(x_train_title)

((0, 3428), 1)
((0, 4950), 1)
((0, 5461), 1)
((0, 5519), 1)
((0, 6861), 1)
((1, 886), 1)
((1, 2957), 1)
((1, 4369), 1)
((1, 4480), 1)
((1, 4593), 1)
((1, 7077), 1)
((1, 7251), 1)
((1, 8028), 1)
((1, 8527), 1)
((1, 8837), 1)
((2, 1142), 1)
((2, 2769), 1)
((2, 2813), 1)
((2, 3079), 1)
((2, 3701), 1)
((2, 3856), 1)
((2, 4276), 1)
((2, 4419), 1)
((2, 4605), 1)
((2, 4846), 1)
((2, 5104), 1)
((2, 5560), 1)
((2, 5819), 1)
((2, 8028), 1)
((2, 8716), 1)
((2, 8797), 1)
((3, 86), 1)
((3, 251), 1)
((3, 466), 1)
((3, 1754), 1)
((3, 2396), 1)
((3, 3246), 1)
((3, 3975), 1)
((3, 5111), 1)
((3, 5208), 1)
((3, 5593), 1)
((3, 5754), 1)
((3, 6641), 1)
((3, 7312), 1)
((3, 7850), 1)
((4, 5935), 1)
((4, 8131), 1)
((5, 86), 1)
((5, 1685), 1)
((5, 4048), 1)
((5, 5712), 1)
((5, 5721), 1)
((5, 8028), 1)
((5, 8150), 1)
((5, 8701), 1)
((6, 2004), 1)
((6, 2606), 1)
((6, 4466), 1)
((6, 5754), 1)
((6, 6526), 1)
((6, 7427), 1)
((6, 8028), 1)
((7, 3729), 1)
((7, 5557), 1)
((7, 7076), 1)
((7, 7143), 1)
((7, 7964), 1)
((

((164, 8685), 1)
((165, 518), 1)
((165, 4668), 1)
((165, 5721), 1)
((165, 6079), 1)
((165, 7955), 1)
((165, 8230), 1)
((166, 617), 1)
((166, 2435), 1)
((166, 3216), 1)
((166, 3503), 1)
((166, 3653), 1)
((166, 5022), 1)
((166, 6409), 1)
((166, 8444), 1)
((167, 1382), 1)
((167, 4642), 1)
((167, 5054), 1)
((167, 5164), 1)
((167, 5712), 1)
((167, 6107), 1)
((167, 8028), 1)
((167, 8726), 1)
((168, 157), 1)
((168, 3567), 1)
((168, 4025), 1)
((168, 5410), 1)
((168, 8028), 1)
((168, 8830), 1)
((168, 8901), 1)
((169, 1944), 1)
((169, 3265), 1)
((169, 3820), 1)
((169, 4949), 1)
((169, 5008), 1)
((169, 6698), 1)
((169, 8028), 1)
((169, 8150), 1)
((170, 441), 1)
((170, 2455), 1)
((170, 3960), 1)
((170, 5338), 1)
((170, 5754), 1)
((170, 6237), 1)
((171, 528), 1)
((171, 904), 1)
((171, 2309), 1)
((171, 2455), 1)
((171, 2684), 1)
((171, 3259), 1)
((171, 4893), 1)
((171, 8531), 1)
((172, 134), 1)
((172, 761), 1)
((172, 789), 1)
((172, 3161), 1)
((172, 3906), 1)
((172, 5034), 1)
((172, 8831), 1)
((173,

((327, 6380), 1)
((327, 8548), 1)
((328, 2749), 1)
((328, 3729), 1)
((328, 5381), 1)
((328, 5721), 1)
((328, 7964), 1)
((328, 8230), 1)
((329, 2289), 1)
((329, 3399), 1)
((329, 4714), 1)
((329, 5461), 1)
((329, 8008), 1)
((329, 8797), 1)
((330, 1488), 1)
((330, 2404), 1)
((330, 4248), 1)
((330, 4454), 1)
((330, 5754), 1)
((330, 5923), 1)
((330, 6975), 1)
((330, 8873), 1)
((331, 4062), 1)
((331, 5027), 1)
((332, 2214), 1)
((332, 2734), 1)
((332, 4226), 1)
((332, 4593), 1)
((332, 6059), 1)
((332, 7015), 1)
((332, 7619), 1)
((332, 8028), 1)
((332, 8636), 1)
((333, 2051), 1)
((333, 3161), 1)
((333, 3293), 1)
((333, 3820), 1)
((333, 3959), 1)
((333, 4546), 1)
((333, 5587), 1)
((333, 5754), 1)
((333, 6154), 1)
((333, 8028), 1)
((333, 8840), 1)
((333, 8942), 1)
((334, 487), 1)
((334, 886), 1)
((334, 923), 1)
((334, 1317), 1)
((334, 1363), 1)
((334, 1688), 1)
((334, 2023), 1)
((334, 2792), 1)
((334, 3073), 1)
((334, 3300), 1)
((334, 4952), 1)
((334, 6750), 1)
((334, 7659), 1)
((334, 7905), 1)


((398, 4470), 1)
((398, 5596), 1)
((398, 7759), 1)
((398, 8531), 1)
((399, 724), 1)
((399, 2169), 1)
((399, 3057), 1)
((399, 3082), 1)
((399, 3456), 1)
((399, 5721), 1)
((399, 6392), 1)
((399, 8531), 1)
((400, 866), 1)
((400, 1649), 2)
((400, 1934), 1)
((400, 3259), 1)
((400, 4783), 1)
((400, 5754), 1)
((400, 6031), 1)
((400, 6719), 1)
((400, 7696), 1)
((401, 1649), 1)
((401, 5183), 1)
((401, 8491), 1)
((402, 529), 1)
((402, 1472), 1)
((402, 5388), 1)
((402, 6439), 1)
((402, 7069), 1)
((402, 7707), 1)
((402, 8028), 1)
((402, 8054), 1)
((402, 8676), 1)
((402, 8787), 1)
((403, 1017), 2)
((403, 3729), 1)
((403, 5281), 1)
((403, 5449), 1)
((403, 5557), 1)
((403, 5721), 1)
((403, 8230), 1)
((404, 1088), 1)
((404, 2982), 1)
((404, 3157), 1)
((404, 4298), 1)
((404, 4383), 1)
((404, 4676), 1)
((404, 4800), 1)
((405, 2918), 1)
((405, 6593), 1)
((405, 7077), 1)
((405, 7367), 1)
((405, 7509), 1)
((405, 7912), 1)
((405, 7955), 1)
((405, 8031), 1)
((405, 8482), 1)
((405, 8798), 1)
((405, 8910), 1)


((524, 305), 1)
((524, 2918), 1)
((524, 3800), 1)
((524, 7040), 1)
((524, 7707), 1)
((524, 8134), 1)
((524, 8482), 1)
((525, 485), 1)
((525, 626), 1)
((525, 1742), 1)
((525, 3588), 1)
((525, 3835), 1)
((525, 4283), 1)
((525, 4518), 1)
((525, 5712), 1)
((525, 6989), 1)
((525, 8156), 2)
((526, 1887), 1)
((526, 3301), 1)
((526, 7400), 1)
((526, 8836), 1)
((526, 8840), 1)
((527, 2607), 1)
((527, 4469), 1)
((527, 6275), 1)
((528, 11), 1)
((528, 84), 1)
((528, 5388), 1)
((528, 5712), 1)
((528, 6421), 1)
((528, 7431), 1)
((528, 8028), 1)
((528, 8472), 1)
((529, 2419), 1)
((529, 4312), 1)
((529, 4324), 1)
((529, 4658), 1)
((529, 6227), 1)
((529, 6236), 1)
((529, 7349), 1)
((530, 584), 2)
((530, 725), 2)
((530, 1777), 1)
((530, 6020), 1)
((530, 6051), 1)
((530, 6054), 1)
((530, 6278), 1)
((530, 8028), 1)
((531, 4942), 1)
((531, 5705), 1)
((531, 5721), 1)
((531, 8230), 1)
((532, 84), 1)
((532, 466), 1)
((532, 663), 1)
((532, 723), 1)
((532, 1683), 1)
((532, 2826), 1)
((532, 3729), 1)
((532, 3942

...

((4618, 5538), 1)
((4618, 5754), 1)
((4618, 6283), 1)
((4618, 8021), 1)
((4618, 8666), 1)
((4618, 8743), 1)
((4619, 256), 1)
((4619, 821), 1)
((4619, 951), 1)
((4619, 1352), 1)
((4619, 1363), 1)
((4619, 2792), 1)
((4619, 3731), 1)
((4619, 4025), 1)
((4619, 4495), 1)
((4619, 5649), 1)
((4619, 5964), 1)
((4619, 7251), 1)
((4619, 7988), 1)
((4619, 8028), 1)
((4619, 8150), 1)
((4619, 8760), 2)
((4620, 1823), 1)
((4620, 2078), 1)
((4620, 2439), 1)
((4620, 3073), 1)
((4620, 7437), 1)
((4621, 1953), 1)
((4621, 3466), 1)
((4621, 3593), 1)
((4621, 3907), 1)
((4621, 5136), 1)
((4621, 5883), 1)
((4621, 7659), 1)
((4622, 2437), 1)
((4622, 4753), 1)
((4622, 5052), 1)
((4622, 5842), 1)
((4622, 8028), 1)
((4622, 8034), 1)
((4622, 8150), 1)
((4623, 487), 1)
((4623, 2923), 1)
((4623, 3804), 1)
((4623, 5101), 1)
((4623, 5942), 1)
((4623, 7068), 1)
((4623, 8122), 1)
((4623, 8797), 1)
((4624, 487), 1)
((4624, 533), 1)
((4624, 4967), 1)
((4624, 5027), 1)
((4624, 5721), 1)
((4624, 7742), 1)
((4624, 8028), 1

((4676, 759), 1)
((4676, 2725), 1)
((4676, 3804), 1)
((4676, 3898), 1)
((4676, 4759), 1)
((4676, 5712), 1)
((4676, 5754), 1)
((4676, 6542), 1)
((4676, 7724), 1)
((4676, 8028), 2)
((4677, 303), 1)
((4677, 698), 1)
((4677, 2426), 1)
((4677, 2815), 1)
((4677, 4756), 1)
((4677, 5308), 1)
((4677, 5372), 1)
((4677, 7406), 1)
((4677, 8049), 1)
((4677, 8452), 1)
((4677, 8726), 1)
((4678, 346), 1)
((4678, 878), 1)
((4678, 1460), 1)
((4678, 1551), 1)
((4678, 2843), 1)
((4678, 4428), 1)
((4678, 6604), 1)
((4678, 6704), 1)
((4678, 8027), 1)
((4678, 8716), 1)
((4678, 8797), 1)
((4679, 1650), 1)
((4679, 2362), 1)
((4679, 2974), 1)
((4679, 3447), 1)
((4679, 6439), 1)
((4679, 8855), 1)
((4680, 1330), 2)
((4680, 1358), 1)
((4680, 3246), 1)
((4680, 4852), 1)
((4680, 5111), 1)
((4680, 5469), 1)
((4680, 5491), 1)
((4680, 5712), 1)
((4680, 7503), 1)
((4680, 7667), 1)
((4680, 7782), 1)
((4680, 8028), 1)
((4680, 8913), 1)
((4681, 386), 1)
((4681, 691), 1)
((4681, 1206), 1)
((4681, 1590), 1)
((4681, 3246), 1)

((4803, 6578), 1)
((4803, 8230), 1)
((4804, 2145), 1)
((4804, 3570), 1)
((4804, 3729), 1)
((4804, 4110), 1)
((4804, 4693), 1)
((4804, 5461), 1)
((4804, 5539), 1)
((4804, 5557), 1)
((4804, 5573), 1)
((4804, 5618), 1)
((4804, 8230), 1)
((4804, 8797), 1)
((4805, 40), 1)
((4805, 1458), 1)
((4805, 2829), 1)
((4805, 4025), 1)
((4805, 4684), 1)
((4805, 5274), 1)
((4805, 5781), 1)
((4805, 8797), 1)
((4805, 8868), 1)
((4806, 246), 1)
((4806, 487), 1)
((4806, 1565), 1)
((4806, 4369), 1)
((4806, 4480), 1)
((4806, 5971), 1)
((4806, 8797), 1)
((4807, 1642), 1)
((4807, 3520), 1)
((4807, 4147), 1)
((4807, 4332), 1)
((4807, 4781), 1)
((4807, 5111), 1)
((4807, 5809), 1)
((4807, 7258), 1)
((4807, 8028), 1)
((4807, 8063), 1)
((4808, 2321), 1)
((4808, 2639), 1)
((4808, 5646), 1)
((4808, 8063), 1)
((4808, 8559), 1)
((4809, 84), 1)
((4809, 466), 1)
((4809, 723), 1)
((4809, 2993), 1)
((4809, 4165), 1)
((4809, 4967), 1)
((4809, 5451), 1)
((4809, 6031), 1)
((4809, 6219), 1)
((4809, 7672), 1)
((4809, 8028), 1)


((4865, 8028), 1)
((4866, 2289), 1)
((4866, 2918), 1)
((4866, 4238), 1)
((4866, 4622), 1)
((4866, 4714), 1)
((4866, 7909), 1)
((4866, 8008), 1)
((4866, 8482), 1)
((4867, 91), 1)
((4867, 414), 1)
((4867, 496), 1)
((4867, 834), 1)
((4867, 1522), 1)
((4867, 3195), 1)
((4867, 3729), 1)
((4867, 5721), 1)
((4867, 8230), 1)
((4868, 487), 1)
((4868, 2112), 1)
((4868, 2115), 1)
((4868, 3966), 1)
((4868, 4748), 1)
((4868, 5007), 1)
((4868, 7626), 1)
((4868, 8028), 1)
((4868, 8797), 1)
((4869, 1126), 1)
((4869, 5721), 1)
((4869, 7494), 1)
((4869, 8150), 1)
((4869, 8230), 1)
((4869, 8910), 1)
((4870, 487), 1)
((4870, 1172), 1)
((4870, 2363), 1)
((4870, 2489), 1)
((4870, 2918), 1)
((4870, 3293), 1)
((4870, 3563), 1)
((4870, 4054), 1)
((4870, 5712), 1)
((4870, 7015), 1)
((4870, 8028), 2)
((4870, 8068), 1)
((4870, 8482), 1)
((4870, 8623), 1)
((4871, 487), 1)
((4871, 1466), 1)
((4871, 2582), 1)
((4871, 2792), 1)
((4871, 3072), 1)
((4871, 3716), 1)
((4871, 4229), 1)
((4871, 4329), 1)
((4871, 5100), 1)


((4988, 3179), 1)
((4988, 3268), 1)
((4988, 4558), 1)
((4988, 5529), 1)
((4988, 5570), 1)
((4989, 2437), 1)
((4989, 4073), 1)
((4989, 5257), 1)
((4989, 5712), 1)
((4989, 8028), 1)
((4990, 84), 1)
((4990, 92), 1)
((4990, 675), 1)
((4990, 2710), 1)
((4990, 4782), 1)
((4990, 4783), 1)
((4990, 4807), 1)
((4990, 5374), 1)
((4990, 5653), 1)
((4990, 6761), 1)
((4990, 6871), 1)
((4990, 6975), 1)
((4990, 7397), 1)
((4990, 7831), 1)
((4990, 8797), 1)
((4991, 2151), 1)
((4991, 4025), 1)
((4991, 4718), 1)
((4991, 5461), 1)
((4991, 7527), 1)
((4991, 8512), 1)
((4992, 245), 1)
((4992, 3911), 1)
((4992, 4556), 1)
((4992, 4624), 1)
((4992, 5461), 1)
((4992, 6703), 1)
((4992, 7079), 1)
((4992, 8750), 1)
((4993, 445), 1)
((4993, 845), 1)
((4993, 3495), 1)
((4993, 3809), 1)
((4993, 4025), 1)
((4993, 4185), 1)
((4993, 4688), 1)
((4993, 5205), 1)
((4993, 5390), 1)
((4993, 5451), 1)
((4993, 5641), 1)
((4993, 7251), 1)
((4993, 8028), 1)
((4993, 8531), 1)
((4993, 8760), 1)
((4993, 8849), 1)
((4994, 416), 1)
(

In [60]:

mapper = DataFrameMapper([
    ('title', CountVectorizer()),
    ('description',CountVectorizer(stop_words='english')),
    ('tags',CountVectorizer())
])
X=mapper.fit_transform(train_data)

(5003, 87466)